<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/rclone_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Install**


In [ ]:
# Cài đặt rClone
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0   9296      0 --:--:-- --:--:-- --:--:--  9282
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/rclone.1  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.71.1-linux-amd64/rclone  [binary]
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/sl...
Processing manual pages under /usr/share/man/sl...
Purging old database entries

In [ ]:
# Danh sách remotes đang có
!rclone listremotes

archive:
onedrive:
googledrive1:
googledrive2:
box:
e5renew:


In [ ]:
# Khôi phục file config
!cp "/content/drive/MyDrive/Colab Notebooks/rclone.conf" /root/.config/rclone/rclone.conf

# ⚙️ **Config remotes**

Modify remotes

In [ ]:
!rclone --version

In [ ]:
# Tạo mới, chỉnh sửa, xóa hoặc kiểm tra remote.
!rclone config

Backup config

In [ ]:
# Backup config
!cp /root/.config/rclone/rclone.conf /content/rclone.conf
!rclone copy -v "/content/rclone.conf" "/content/drive/MyDrive/Colab Notebooks/backup/rclone-renamed.conf"

# 🎯 **Các lệnh Copy**

In [ ]:
# @title Tạo tên bucket (S3/GCS-safe)
import re
import secrets
import unicodedata
from datetime import datetime

# ---------- Core helpers ----------
def _to_ascii_slug(s: str) -> str:
    s = (s or "").replace("Đ", "D").replace("đ", "d")
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.encode("ascii", "ignore").decode("ascii").lower()
    s = re.sub(r"[^a-z0-9]+", "-", s)
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s or "bucket"

def _now_in_tz(tz: str):
    try:
        from zoneinfo import ZoneInfo
        return datetime.now(ZoneInfo(tz))
    except Exception:
        return datetime.utcnow()

_S3_BAD_PREFIXES = ("xn--", "sthree-", "amzn-s3-demo-")
_S3_BAD_SUFFIXES = ("-s3alias", "--ol-s3", ".mrap", "--x-s3", "--table-s3")

def _guard_s3(name: str) -> str:
    n = name
    if n.startswith(_S3_BAD_PREFIXES):
        n = "a-" + n
    if n.endswith(_S3_BAD_SUFFIXES):
        n = n + "-a"
    return n

def _guard_gcs(name: str):
    warns = []
    n = name
    if n.startswith("goog"):
        n = "a-" + n
        warns.append("Đã sửa để không bắt đầu bằng 'goog' (GCS cấm).")
    patterns = [r"google", r"g00gle"]
    for pat in patterns:
        if re.search(pat, n):
            n = re.sub(pat, "gcloud", n)
            warns.append("Đã thay 'google/g00gle' -> 'gcloud' (GCS cấm).")
    return n, warns

def _ensure_edges_and_length(name: str, min_len=3, max_len=63) -> str:
    n = name.strip("-")
    if not n or not n[0].isalnum():
        n = "a" + n
    if not n[-1].isalnum():
        n = n + "a"
    if len(n) < min_len:
        n = (n + "a" * (min_len - len(n)))[:min_len]
    if len(n) > max_len:
        n = n[:max_len].rstrip("-")
        if len(n) < min_len:
            n = (n + "a" * (min_len - len(n)))[:min_len]
    return n

# ---------- Public API ----------
def make_bucket_name(
    raw_input: str,
    tz: str = "Asia/Ho_Chi_Minh",
    max_len: int = 63,
    add_unique: bool = True,
    unique_len: int = 4,
    date_format: str = "%Y%m%d",
    provider: str = "s3",
):
    base = _to_ascii_slug(str(raw_input))
    uniq = "".join(secrets.choice("abcdefghijklmnopqrstuvwxyz0123456789")
                   for _ in range(unique_len)) if add_unique and unique_len > 0 else ""
    stamp = _now_in_tz(tz).strftime(date_format)
    suffix = f"{uniq}-{stamp}" if uniq else stamp

    reserved = len(suffix) + 1
    if max_len <= reserved:
        base = "a"
    else:
        base = (base[: (max_len - reserved)]).strip("-") or "a"

    name = f"{base}-{suffix}"
    if provider in ("s3", "both"):
        name = _guard_s3(name)
    warnings = []
    if provider in ("gcs", "both"):
        name, w = _guard_gcs(name)
        warnings.extend(w)
    name = _ensure_edges_and_length(name, 3, max_len)
    return name, warnings

# ---------- Chạy trực tiếp ----------
raw_input = input("Nhập tên gốc: ")
bucket, warns = make_bucket_name(raw_input)
print("✅ Tên bucket:", bucket)
if warns:
    print("⚠️ Cảnh báo:", *warns, sep="\n - ")


In [ ]:
# @title Basic: Copy files between Cloud
# @markdown **Lưu ý**:
# @markdown - nếu thêm `/` ở cuối, chỉ copy nội dung bên trong
# @markdown - không dùng `/` ở cuối thì copy cả folder

source_remote = "e5renew" # @param ["/content/","googledrive1","e5renew","googledrive2","archive", "box"]
source_folder = "Temps\\[Weekly Playboy] 2021 No.50\\mariya nishiuchi\\up\\tsubaki-sannomiya304" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc file, để trống là toàn bộ remote"}
# @markdown ---

destination_remote = "archive" # @param ["archive","e5renew","googledrive2","googledrive1", "box","/content/"]
destination_folder = "WeeklyPlayboy2021No.50/tsubaki-sannomiya304" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc tạo thư mục trong archive."}
# @markdown ---

option_flag = "" # @param {"type":"string","placeholder":"thêm `--flag` nếu cần, dùng space phân cách các flag. Để trống để auto-detect"}

# Kiểm tra và thực thi lệnh copy
if source_remote and destination_remote:
    # FIX: Chuyển tất cả backslash thành forward slash
    source_folder = source_folder.replace('\\', '/') if source_folder else ""
    destination_folder = destination_folder.replace('\\', '/') if destination_folder else ""

    # Xử lý đường dẫn SOURCE
    if source_remote.startswith("/"):
        source_path = f"{source_remote}{source_folder}" if source_folder else source_remote
    else:
        source_path = f"{source_remote}:{source_folder}" if source_folder else f"{source_remote}:"

    # Xử lý đường dẫn DESTINATION
    if destination_remote.startswith("/"):
        destination_path = f"{destination_remote}{destination_folder}" if destination_folder else destination_remote
    else:
        destination_path = f"{destination_remote}:{destination_folder}" if destination_folder else f"{destination_remote}:"

    # AUTO-DETECT FLAGS hoặc dùng manual
    if option_flag.strip():
        extra_flags = option_flag.strip()
    else:
        if destination_remote == "archive":
            extra_flags = "--s3-upload-concurrency 1"
        else:
            extra_flags = "--transfers 10"

    print(f"🚀 Command: rclone copy -v {extra_flags} \"{source_path}\" \"{destination_path}\"")

    # Chạy lệnh rclone
    !rclone copy -v {extra_flags} "{source_path}" "{destination_path}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote.")

🚀 Command: rclone copy -v --s3-upload-concurrency 1 "e5renew:Temps/[Weekly Playboy] 2021 No.50/mariya nishiuchi/up/tsubaki-sannomiya304" "archive:WeeklyPlayboy2021No.50/tsubaki-sannomiya304"
2025/10/04 11:23:07 INFO  : gra_tsubaki-sa3_062.jpg: Copied (new)
2025/10/04 11:23:07 INFO  : gra_tsubaki-sa3_064.jpg: Copied (new)
2025/10/04 11:23:10 INFO  : gra_tsubaki-sa3_066.jpg: Copied (new)
2025/10/04 11:23:10 INFO  : gra_tsubaki-sa3_065.jpg: Copied (new)
2025/10/04 11:23:11 INFO  : gra_tsubaki-sa3_063.jpg: Copied (new)
2025/10/04 11:23:11 INFO  : gra_tsubaki-sa3_061.jpg: Copied (new)
2025/10/04 11:23:12 INFO  : gra_tsubaki-sa3_067.jpg: Copied (new)
2025/10/04 11:23:13 INFO  : gra_tsubaki-sa3_069.jpg: Copied (new)
2025/10/04 11:23:13 INFO  : gra_tsubaki-sa3_068.jpg: Copied (new)
2025/10/04 11:23:13 INFO  : gra_tsubaki-sa3_070.jpg: Copied (new)
2025/10/04 11:23:15 INFO  : gra_tsubaki-sa3_071.jpg: Copied (new)
2025/10/04 11:23:16 INFO  : gra_tsubaki-sa3_073.jpg: Copied (new)
2025/10/04 11:23:



---



In [ ]:
!rclone lsf "archive:1okrock-liveshow"

In [ ]:
!rclone copyurl -v "https://c1.cdnjav.com/content-01/contents/3-lafbd-026-ruka-kanae-laforet-girl-26/videos/3-lafbd-026-ruka-kanae-laforet-girl-26_sh.mp4" "archive:thePaintedBird2019" --s3-upload-concurrency 1

In [ ]:
!wget -P /content/temp/ "https://c1.cdnjav.com/content-01/contents/3-lafbd-026-ruka-kanae-laforet-girl-26/videos/3-lafbd-026-ruka-kanae-laforet-girl-26_sh.mp4"

### 🍷 **Ghi chú**

**Command**
* `copy` : Copy file/thư mục từ nguồn tới đích, bỏ qua file giống nhau
* `copyurl` : Copy direct link
* `check` : So sánh nội dung giữa nguồn và đích
* `sync` : đồng bộ (dùng --dry-run kiểm tra trước khi chạy thật).
* `ls`: Liệt kê file và kích thước (mặc định).
* `lsf`: Chỉ hiển thị tên file/thư mục.
* `lsd`: Chỉ hiển thị tên thư mục con.
* `size`: Hiển thị tổng kích thước của thư mục.

---
**Flag**

* Ghi đè nếu nguồn mới hơn: `--update`. (không dùng được trên archive)
* Bỏ qua các file đã tồn tại: `--ignore-existing`. (phù hợp với *archive.org* để tránh gây lỗi, còn lại thì dùng mặc định `copy` là đủ).
*
 * Chọn số luồng copy (nếu không dùng thì mặc định là 4)): `--transfers 8`.
 * Dùng cho S3 của Amazon: `--s3-upload-concurrency 1`. (archive.org file lớn ưu tiên dùng ✅ 1 luồng, tối đa dùng được 2 luồng).

* `--dry-run` : Chạy thử, kiểm tra trước.

* `--include "*.mp4"` để chỉ copy file .mp4.
* `--exclude "*.tmp`" để bỏ qua file tạm.
 * Ex lọc files: `!rclone ls "archive:cd-luong-bich-huu" --include "*.flac"`
 * Ex 2: `rclone copy source dest --include "*.jpg" --exclude "*.gif"`

* `--error string` : Báo cáo files lỗi

# 👓 **Xem Nội Dung Remote**

In [ ]:
# @markdown ### Hướng dẫn:
# @markdown - **List Mode**: Chọn cách hiển thị nội dung:
# @markdown   - `ls`: Liệt kê file và kích thước (mặc định).
# @markdown   - `lsf`: Chỉ hiển thị tên file/thư mục.
# @markdown   - `lsd`: Chỉ hiển thị tên thư mục con.
# @markdown   - `size`: Hiển thị tổng kích thước của thư mục.

list_remote = "box" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive","box"]
input_folder = "" # @param {"type":"string"}
list_mode = "lsf" # @param ["lsf","lsd","ls","size"]
# @markdown 🔎 Lọc file:
file_type = "" # @param {"type":"string"}
file_type_sum = "Default" # @param ["Default", "Sum"]

# Xử lý file_type_sum thành lệnh phù hợp
sum_command = "" if file_type_sum == "Default" else " | wc -l"


!rclone {list_mode} "{list_remote}:{input_folder}"  | grep ".{file_type}"  {sum_command}


# ======================================
# grep ".mp4": Lọc các dòng có chứa .mp4.
# wc -l: Đếm số dòng (tức là số file .mp4).

# 🗃️ **.Rar .Zip .7zip**

## Rar

In [ ]:
# Install unrar
!apt-get install unrar

In [ ]:
# Giải nén với password (ex: DDDTT)
!unrar x -pdomdom2010 /content/copy_temp/*.zip /content/copy_temp/extracted/

## Zip

In [ ]:
# Install unzip
!apt-get install -y unzip

In [ ]:
# Giải nén file .zip với password (nếu có)
!unzip -P kirakira /content/copy_temp/Ed01.zip -d /content/copy_temp/extracted/

In [ ]:
# Dùng vòng lặp `for` Giải nén nhiều file .zip trong thư mục với password (nếu có)
import os
for zip_file in os.listdir("/content/copy_temp/2/"):
    if zip_file.endswith(".zip"):
        !unzip -P domdom2010 "/content/copy_temp/2/{zip_file}" -d /content/copy_temp/2/s2/
        print(f"Đã giải nén {zip_file}")

## 7-zip

In [ ]:
# Cài đặt p7zip
!apt-get update
!apt-get install -y p7zip-full

In [ ]:
# Giải nén file multi-part 7z
!7z x "/content/video/【第九期】tenten人体结构&光影基础课.7z.001"

---
## uncheck

rar

In [ ]:
# hình như là nén files, hoặc ghép files. cần thử lại.
!cat /content/torrent_download/HI1\ DDDTT.part*.rar > /content/HI1_DDDTT_full.rar

In [ ]:
# Split file nén
!split -b 10G /content/HI1_DDDTT_full.rar /content/HI1_DDDTT_part_
# copy file trùng tên, chhia theo thứ tự.
!rclone copy /content/HI1_DDDTT_part_* archive:tinhhe-lontien-thyetmin/1990/ -v

7zip
* p7zip-full hỗ trợ hầu hết định dạng (.7z, .zip, .rar, .iso, .tar, .gzip, v.v.), nhưng hiệu suất giải nén .rar tốt hơn với unrar.

In [ ]:
!apt-get update
!apt-get install -y p7zip-full

In [ ]:
!7z x /content/file.7z -o/content/extracted/ -pYOUR_PASSWORD

In [ ]:
!7z x /content/file.iso -o/content/extracted/

In [ ]:
import os

# Đường dẫn thư mục chứa file nén
source_folder = "/content/copy_temp/2/"
extract_folder = "/content/copy_temp/2/zip2/"
password = "domdom2010"  # Thay bằng password thực tế

# Tạo thư mục đích nếu chưa có
!mkdir -p {extract_folder}

# Lặp qua các file trong thư mục
for filename in os.listdir(source_folder):
    if filename.endswith(".zip"):
        file_path = os.path.join(source_folder, filename)
        # Sử dụng 7z để giải nén
        !7z x "{file_path}" -o"{extract_folder}" -p{password} -y
        if os.path.exists(os.path.join(extract_folder, filename.replace(".zip", ""))):
            print(f"Đã giải nén {filename} thành công")
        else:
            print(f"Lỗi: Không thể giải nén {filename}. Kiểm tra file hoặc password!")

cai đặt cả 3

In [ ]:
# Cài đặt các công cụ giải nén
!apt-get update
!apt-get install -y unrar unzip p7zip-full

# 📺 **yt-dlp**

## ⚙️ Install yt-dlp & ffmpeg
* ⚠️Chú ý dùng ! trước command
* Nếu lỗi 403 Forbidden, thêm --geo-bypass

In [ ]:
pip install -U yt-dlp

In [ ]:
!sudo apt install ffmpeg

## ⏬ Hướng dẫn


*   `!yt-dlp`: không dùng flag thì tự chọn định dạng tốt nhất
*   `-v`: để xem log
*   Subtitle:
 *   `--write-subs --sub-lang en`: tải 1 hoặc nhiều subtitles `en,vi,kr`. (ex: *en = english*)
 *   `--all-subs`: tải file và tất cả subtitles.
 *   `--list-subs`: danh sách các subtitles.
 *   `--skip-download`: bỏ qua không tải files, nếu chỉ tải subtitle.

*   `-F`: xem tất cả định dạng
*   `-f`: chọn tải định dạng (ex: `-f 251`)

*   `--get-urlurl`: tải .m3u8

*   `-o`: đặt tên file.
*   `-P`: path đường dẫn để lưu file.

---
`--postprocessor-args "-ss 00:00 -to 00:55"`: cắt âm thanh <~ chua test


## command

In [ ]:
# Tải phim Đường triêu quỷ sự lục 2
!yt-dlp --write-sub --sub-lang vi "https://youtu.be/Acmx5EP17wA" -P "phim"

In [ ]:
!yt-dlp -f "bestaudio" \
  -o "/content/drive/MyDrive/audio_downloads/%(upload_date)s_%(title)s.%(ext)s" \
  --ignore-errors \
  --user-agent "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36" \
  --sleep-interval 1 \
  --max-sleep-interval 3 \
  "https://www.youtube.com/@vuogle/streams"

# 🎦 Upload video from colab to youtube

1) Mấy lưu ý quan trọng (đừng bỏ qua)
Upload video bắt buộc dùng OAuth 2.0 (không dùng API key). Các “insert/update/delete” đều cần token ủy quyền của người dùng YouTube. 12
Service Account không dùng để upload YouTube (YouTube không hỗ trợ flow này vì không gắn được channel). Hãy dùng OAuth của tài khoản người dùng và lưu refresh token để tái sử dụng. 3
Quota: Gọi videos.insert tốn 1600 đơn vị; quota mặc định của dự án là 10.000 đơn vị/ngày (reset theo Pacific Time). Nghĩa là mỗi ngày bạn chỉ up được vài video bằng API nếu không xin tăng quota. 24
Giới hạn file: tối đa 256GB, MIME video/* hoặc application/octet-stream. 2
Dự án chưa “verified” (tạo sau 28/07/2020) khi upload bằng API, video có thể bị auto-Private đến khi dự án được audit/verify. 2
2) Tạo dự án và bật YouTube Data API v3
Vào Google Cloud Console → tạo project mới. 5
Vào APIs & Services → Library → tìm YouTube Data API v3 → Enable. 5
Vào APIs & Services → OAuth consent screen:
Chọn loại “External” (nếu chỉ mình bạn dùng thì để “Testing” và add email của bạn vào Test users).
Điền các thông tin cơ bản của app. (Bước này là bắt buộc để xin quyền upload). 3
Vào Credentials → Create Credentials → OAuth client ID → Desktop app → Create → Download JSON (thường tên client_secret_*.json). 6
Vì upload là thao tác ghi, bạn sẽ xin scope https://www.googleapis.com/auth/youtube.upload (hoặc rộng hơn nếu cần). 2

3) Chuẩn bị trong Colab
Upload token.json + client_secret.json lên /content của Colab.
File video của bạn nằm ở đâu? Ví dụ /content/video.mp4 (bộ nhớ tạm Colab) hoặc /tmp/clip.mp4.



```
# Chay o may local (Windows) de xac thuc va tao token.json

# get_token_local.py  (Windows/Mac/Linux)
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]  # quyền upload
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", SCOPES)

# Dùng loopback server. Nếu cần, bạn đổi port cố định 8765 để tránh firewall
creds = flow.run_local_server(host="localhost", port=8765, prompt="consent", open_browser=True)

with open("token.json", "w", encoding="utf-8") as f:
    f.write(creds.to_json())

print("✅ Đã tạo token.json. Bạn upload file này lên Colab nhé.")
```



In [ ]:
# 1. Ham Xác thực và tạo YouTube client
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import os

SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
TOKEN_FILE = "/content/token.json"
CLIENT_SECRET_FILE = "/content/client_secret.json"

def get_youtube_client():
    if not os.path.exists(TOKEN_FILE):
        raise RuntimeError("Thiếu token.json. Hãy tạo trên Windows và upload lên /content.")

    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds.valid:
        if creds.expired and creds.refresh_token:
            creds.refresh(Request())
            with open(TOKEN_FILE, "w") as f:
                f.write(creds.to_json())
        else:
            raise RuntimeError("Token không hợp lệ và không refresh được. Tạo lại token.json.")
    return build("youtube", "v3", credentials=creds)

yt = get_youtube_client()
print("YouTube client sẵn sàng! ✅")

YouTube client sẵn sàng! ✅


In [ ]:
# 2. Ham upload video
import math, time

def upload_video(
    file_path,
    title="Demo upload từ Colab",
    description="Upload qua YouTube Data API",
    tags=None,
    category_id="22",           # 22 = People & Blogs
    privacy_status="unlisted",  # 'public' | 'unlisted' | 'private'
    notify_subscribers=True     # có gửi thông báo sub hay không
):
    youtube = yt  # dùng client đã tạo

    body = {
        "snippet": {
            "title": title,
            "description": description,
            "tags": tags or [],
            "categoryId": category_id
        },
        "status": {
            "privacyStatus": privacy_status
        }
    }

    # Resumable upload: chunk 8MB (bạn có thể tăng nếu mạng ổn định)
    media = MediaFileUpload(file_path, chunksize=8 * 1024 * 1024, resumable=True)

    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media,
        notifySubscribers=notify_subscribers
    )  # notifySubscribers là tham số hợp lệ của videos.insert [2](https://developers.google.com/youtube/v3/docs/videos/insert)

    print(f"Bắt đầu upload: {file_path}")
    response = None
    last_print = 0
    while response is None:
        status, response = request.next_chunk()  # vòng lặp chunk theo hướng dẫn của Google [7](https://github.com/googleapis/google-api-python-client/blob/main/docs/media.md)
        if status:
            pct = int(status.progress() * 100)
            if pct - last_print >= 5:  # in mỗi 5%
                print(f"Tiến độ: {pct}%")
                last_print = pct

    video_id = response.get("id")
    print(f"Hoàn tất! Video ID: {video_id}")
    return video_id

# Ví dụ: đường dẫn trong bộ nhớ tạm của Colab
video_path = "/content/【第九期】tenten人体结构&光影基础课/3 第一周 作业评讲.mp4"  # hoặc "/tmp/clip.mp4"
video_id = upload_video(
    file_path=video_path,
    title="Test upload từ Colab",
    description="Demo bằng YouTube Data API v3",
    tags=["colab", "api", "demo"],
    privacy_status="unlisted"
)

Bắt đầu upload: /content/【第九期】tenten人体结构&光影基础课/3 第一周 作业评讲.mp4
Tiến độ: 5%
Tiến độ: 10%
Tiến độ: 15%
Tiến độ: 20%
Tiến độ: 26%
Tiến độ: 31%
Tiến độ: 36%
Tiến độ: 41%
Tiến độ: 46%
Tiến độ: 51%
Tiến độ: 56%
Tiến độ: 61%
Tiến độ: 66%
Tiến độ: 71%
Tiến độ: 76%
Tiến độ: 81%
Tiến độ: 86%
Tiến độ: 91%
Tiến độ: 96%
Hoàn tất! Video ID: 4qdAnTVoznI


# 🚄 Torrent


*   ⚠️ Không nên dùng vì vi phạm chính sách Colab



In [ ]:
# @title 📌 Install torrent
#  install
!pip install lbry-libtorrent

Nếu không được thì thử cài libtorrent

```
# Kiểm tra phiên bản Python
import sys; print(sys.version)
# Nếu Python 3.12: thử bản chính thức mới
!pip -q install -U "libtorrent>=2.0.11"
```

In [ ]:
# @title ⏬ Dowload Torent
import os, time, libtorrent as lt

SAVE_PATH = "/content/torrent_download"
os.makedirs(SAVE_PATH, exist_ok=True)

def display_progress(h):
    s = h.status()
    state_str = [
        'queued', 'checking', 'downloading metadata', 'downloading',
        'finished', 'seeding', 'allocating', 'checking fastresume'
    ]
    state = state_str[s.state] if 0 <= s.state < len(state_str) else f"state:{s.state}"
    return (f"Tiến độ: {s.progress*100:.2f}% | Down: {s.download_rate/1024:.1f} KB/s | "
            f"Up: {s.upload_rate/1024:.1f} KB/s | Peer: {s.num_peers} | {state}")

def list_files(h):
    ti = h.get_torrent_info()
    fs = ti.files()
    n = fs.num_files()
    return [(i, fs.file_path(i), fs.file_size(i)) for i in range(n)]

def file_progress_bytes(h):
    # v2: trả về list bytes đã tải cho từng file
    try:
        return h.file_progress()
    except TypeError:
        return h.file_progress(0)

# --- session ---
ses = lt.session()
try:
    ses.listen_on(6881, 6891)
except Exception:
    pass
ses.start_dht()

# --- add magnet ---
magnet = input("Nhập magnet (q để thoát): ").strip()
if magnet.lower() == 'q':
    raise SystemExit

# thêm flag sequential cho ổn định streaming (tùy chọn)
atp = lt.parse_magnet_uri(magnet)
atp.save_path = SAVE_PATH
if hasattr(lt.torrent_flags, "sequential_download"):
    atp.flags |= lt.torrent_flags.sequential_download

# (tùy chọn) thêm tracker để bắt metadata nhanh
atp.trackers = [
    "udp://tracker.opentrackr.org:1337/announce",
    "udp://open.stealth.si:80/announce",
    "udp://tracker.openbittorrent.com:6969/announce",
]

h = ses.add_torrent(atp)

# --- chờ metadata ---
t0 = time.time()
while not h.has_metadata():
    if time.time() - t0 > 90:
        print("Timeout lấy metadata (ít peer/seeder).")
        raise SystemExit
    time.sleep(1)
print("Đã lấy metadata.")

# --- liệt kê file ---
files = list_files(h)
for i, p, sz in files:
    print(f"[{i}] {p} ({sz/1024/1024:.2f} MB)")

# --- chọn tải ---
choice = input("Tải toàn bộ (y) hay chọn file (n)? ").strip().lower()
selected = None
if choice == 'n':
    raw = input("Nhập index, ví dụ 0,2,4: ")
    idxs = []
    for x in raw.split(','):
        x = x.strip()
        if x.isdigit() and 0 <= int(x) < len(files):
            idxs.append(int(x))
    selected = sorted(set(idxs))

# --- đặt ưu tiên file ---
num_files = len(files)
if selected:
    prio = [0]*num_files
    for i in selected: prio[i] = 1
else:
    prio = [1]*num_files

try:
    h.prioritize_files(prio)
except AttributeError:
    for i, p in enumerate(prio):
        h.file_priority(i, p)

print("Bắt đầu tải...")
last = 0
while True:
    s = h.status()
    now = time.time()
    if now - last >= 5:
        print(display_progress(h))
        if selected:
            prog = file_progress_bytes(h)
            for i in selected:
                done = prog[i] if i < len(prog) else 0
                size = files[i][2]
                pct = (done/size*100) if size else 0
                print(f"  - [{i}] {files[i][1]}: {pct:.2f}% ({done/1024/1024:.2f}/{size/1024/1024:.2f} MB)")
        last = now
    if s.state in (lt.torrent_status.seeding, lt.torrent_status.finished):
        print("Hoàn tất.")
        break
    time.sleep(1)

# liệt kê file kết quả
for root, _, fns in os.walk(SAVE_PATH):
    for f in fns:
        p = os.path.join(root, f)
        print("- ", p)


# JavaScript giữ kết nối colab


```
# F12 > Console
function ClickConnect(){
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 300000); // 5 phút
```





---
# lung tung chua test 👇


## Tải audio (mp3) cho 5 link cùng lúc

*   Chỉ cần thay LINK_1 … LINK_5 bằng 5 link của bạn (có thể dán xuống dòng cũng được).

```
!yt-dlp -x --audio-format m4a --audio-quality 0 \
  --embed-thumbnail --add-metadata \
  -o "%(title)s.%(ext)s" \
  "LINK_1" "LINK_2" "LINK_3" "LINK_4" "LINK_5"
  -P "/content/drive/MyDrive/YoutubeAudio"
```


